In [7]:
import pandas as pd
import yfinance as yf

In [9]:
# Obtener datos históricos del S&P 500 (últimos 100 días)
sp = pd.read_csv('S&P500-2025-01-10.csv', index_col='Date')
sp = sp.loc['2020-01-01':'2025-01-01']

In [ ]:
horizons = [2, 5, 60, 250, 1000]    # Horizontes de tiempo como mencionaste
new_predictors = []  # Lista de nuevos predictores

# Calcular los nuevos predictores
for horizon in horizons:
    rolling_averages = sp.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    sp[ratio_column] = sp["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    sp[trend_column] = sp.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors+= [ratio_column, trend_column]

# Mostrar las nuevas características creadas
print(sp[new_predictors].tail())

                           Close_Ratio_2  Trend_2  Close_Ratio_5  Trend_5  \
Date                                                                        
2024-12-24 05:00:00+00:00       1.005491      2.0       1.017383      3.0   
2024-12-26 05:00:00+00:00       0.999797      1.0       1.011334      3.0   
2024-12-27 05:00:00+00:00       0.994441      0.0       0.996689      3.0   
2024-12-30 05:00:00+00:00       0.994620      0.0       0.986810      2.0   
2024-12-31 05:00:00+00:00       0.997853      0.0       0.985626      1.0   

                           Close_Ratio_60  Trend_60  Close_Ratio_250  \
Date                                                                   
2024-12-24 05:00:00+00:00        1.023002      35.0         1.115492   
2024-12-26 05:00:00+00:00        1.021639      35.0         1.114008   
2024-12-27 05:00:00+00:00        1.009600      34.0         1.100716   
2024-12-30 05:00:00+00:00        0.998213      34.0         1.088002   
2024-12-31 05:00:00+00:00   

In [11]:
# Seleccionar las columnas generadas como predictores
X_pred = sp[new_predictors].dropna()  # Eliminar valores nulos

# Asegúrate de que las predicciones solo se hagan sobre los datos que contienen características completas

In [12]:
import joblib

# Cargar el modelo previamente entrenado
rf_model = joblib.load('modelo_random_forest.pkl')

# Realizar las predicciones para los últimos 100 días
predicciones = rf_model.predict(X_pred)

# Ver las predicciones
print(predicciones)

[1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 0 1 1
 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1
 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0
 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 1]


In [14]:
len(predicciones)

258

In [16]:
len(sp)

1258

In [17]:
# Inicializamos el capital y otras variables
capital = 1  # Empezamos con $1
comprado = False
precio_compra = 0

# Simulación de las decisiones de inversión basadas en las predicciones
for i in range(1, len(X_pred)):
    prediccion = predicciones[i-1]  # Predicción del modelo para el día anterior
    precio_actual = sp["Close"].iloc[i]

    if prediccion == 1 and not comprado:  # Si el modelo predice que subirá y no hemos comprado
        comprado = True
        precio_compra = precio_actual
    elif prediccion == 0 and comprado:  # Si el modelo predice que bajará y ya hemos comprado
        capital *= (precio_actual / precio_compra)  # Rentabilidad de la inversión
        comprado = False

# Si al final seguimos con la inversión, vendemos al final
if comprado:
    capital *= (sp["Close"].iloc[-1] / precio_compra)

print(f"Valor final siguiendo el modelo: ${capital:.2f}")


Valor final siguiendo el modelo: $0.82


In [18]:
# Calcular el rendimiento de la inversión pasiva
precio_inicio = sp["Close"].iloc[0]
precio_final = sp["Close"].iloc[-1]
rendimiento_sp500 = (precio_final / precio_inicio) * 1

# Comparar ambos resultados
print(f"Valor de la inversión pasiva: ${rendimiento_sp500:.2f}")
print(f"Valor siguiendo el modelo: ${capital:.2f}")


Valor de la inversión pasiva: $1.81
Valor siguiendo el modelo: $0.82
